In [20]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import datasets
from glob import glob
import os
import json

# command used
```bash
python /data/BirdNET-Analyzer/analyze.py --i /data/tests/generated/${expname} --o /data/tests/analysed/${expname} --week -1 --lat -1 --lon -1 --rtype "csv" --threads 16
```
note /data/BirdNET-Analyzer is a clone of the repo at https://github.com/kahst/BirdNET-Analyzer

In [5]:
table_file = "/data/tests/stem_table.json"
with open(table_file) as f:
    table = json.load(f)

In [8]:
test_dir = "/data/tests/analysed/steps100_scale7,5_v3"

assert os.path.isdir(test_dir), "not analysed yet"

def read_csv_results_file(file: str):
    df = pd.read_csv(os.path.join(test_dir, file))
    df = df.rename(columns={"Scientific name": "detected_sci_name",
                            "Common name": "common_name",
                            "Confidence": "confidence"})
    df = df.drop(columns=["Start (s)", "End (s)"])
    stem = file.removesuffix(".BirdNET.results.csv")
    df["file"], df["sci_name"] = table[stem]
    return df
    
dfs = filter(lambda df: df.size > 0, [read_csv_results_file(file) for file in tqdm(glob("*.csv", root_dir=test_dir))])
df = pd.concat(dfs, ignore_index=True)
df.head()

100%|██████████| 1000/1000 [00:01<00:00, 609.11it/s]


,detected_sci_name,common_name,confidence,file,sci_name
0,Euplectes ardens,Red-collared Widowbird,0.1796,Poecile palustris/XC189461.wav,Poecile palustris
1,Melanodera xanthogramma,Yellow-bridled Finch,0.1115,Poecile palustris/XC189461.wav,Poecile palustris
2,Poecile sclateri,Mexican Chickadee,0.8399,Poecile palustris/XC189461.wav,Poecile palustris
3,Diglossa cyanea,Masked Flowerpiercer,0.2478,Poecile palustris/XC189461.wav,Poecile palustris
4,Cisticola juncidis,Zitting Cisticola,0.1043,Poecile palustris/XC189461.wav,Poecile palustris


In [9]:
df.to_csv(test_dir+".csv", index=False)

### Test birdnet labels

In [4]:
with open("/data/meta/BirdNET_GLOBAL_6K_V2.4_Labels_da.txt") as f:
    birdnet_labels_str = f.read()

sci_names = xeno_canto.with_format("pandas")["file"].map(lambda x: Path(x).parent.name)
sci_names_unique = pd.Series(sci_names.unique())

In [29]:
is_contained = sci_names.map(lambda x: x in birdnet_labels_str)
print(is_contained.mean())

print("unique")
is_contained = sci_names_unique.map(lambda x: x in birdnet_labels_str)
print(is_contained.mean())

0.8874694684312974
unique
0.5969604263298135


In [20]:
a = birdnetlib.species.SpeciesList()
len(a.return_list())

Labels loaded.
load_species_list_model
Meta model loaded.
-1
6522 species loaded.


6522